In [1]:
import mlflow
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema, ColSpec, ParamSchema, ParamSpec
import requests

class MyModel(mlflow.pyfunc.PythonModel):
    def predict(self, ctx, model_input, params):
        flower_name = model_input['word'][0]
        return {"word":flower_name.upper() if params['do_work'] else flower_name}

In [3]:
params_schema = ParamSchema(
    [
        ParamSpec("do_work", "boolean", False)
    ]
)

input_schema = Schema(
    [
        ColSpec("string", "word")
    ]
)

output_schema = Schema(
    [
        ColSpec("string","word")
    ]
)

signature = ModelSignature(inputs=input_schema,
                           outputs=output_schema,
                           params = params_schema)

mlflow.set_experiment("upper_model")
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        python_model=MyModel(),
        artifact_path="upper_model",
        signature=signature,
        registered_model_name="upper_model" # Register the model
    )

client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions("upper_model", stages=["None"])
latest_model_version = model_metadata[0].version

2024/01/26 16:45:13 INFO mlflow.tracking.fluent: Experiment with name 'upper_model' does not exist. Creating a new experiment.
Successfully registered model 'upper_model'.
2024/01/26 16:45:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: upper_model, version 1
Created version '1' of model 'upper_model'.


In [7]:
mlflow.models.get_model_info(f"models:/upper_model/{latest_model_version}").signature

inputs: 
  ['word': string]
outputs: 
  ['word': string]
params: 
  ['do_work': boolean (default: False)]

# Usage

In the following are two ways to use the registered models. 

## Loading the model

Using pyfunc.load_model you can test if your model is working well, but it doesn`t involves nothing of the model services feature.

In order to use it, change the version of model_uri to the latests version you deployed

In [6]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/upper_model/{latest_model_version}")
print(model.predict({"word":"hello world"}, params={"do_work":True}))
print(model.predict({"word":"hello world"}, params={"do_work":False}))

{'word': 'HELLO WORLD'}
{'word': 'hello world'}


## Model services

Go to the published services page and create a model, run it and wait until the running state. Change the URL field below to the url of the published services.

In [5]:
# Model service 🤘

url = "https://10.137.137.5:5000"+"/invocations"
data = {
    "inputs":{
        "word":["hello world"],
    },
    "params":{
        "do_work": True
    }
}

headers = {'Content-type': 'application/json'}
x = requests.post(url, json = data, verify=False, headers=headers)

print(x.text)

{"predictions": {"flowerName": "sunflower", "do_work": false}}


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.137.137.5'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
